In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from os.path import join
from os import getcwd
from IPython.display import clear_output

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix, auc

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD, Adam
from keras.metrics import AUC # Area under the curve, default: ROC
from keras.losses import BinaryCrossentropy
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.optimizers.schedules import ExponentialDecay
from keras.initializers import GlorotNormal
import kerastuner as kt

In [4]:
from functions import *

In [5]:
checkpoints_path = getcwd()+'\\checkpoints'

In [6]:
df = pd.read_csv('../../databases/diabetes.csv')
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [7]:
outlayers = {
    'BloodPressure': (40, np.Inf),
    'SkinThickness': (0, 80),
    'Insulin': (0, 400),
    'BMI': (0, 50)
}

zeros = [
    'Glucose',
    'BloodPressure',
    'SkinThickness',
    'Insulin',
    'BMI'
]
x_df = df[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']]
y_df = df['Outcome']

In [8]:
# Split dataset into 15% test, 85% train 
x_temp, x_test, y_temp, y_test = train_test_split(x_df, y_df, test_size=0.15, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_temp, y_temp, test_size=0.15, random_state=0)

# Regresión Logistica

### Con los datos sin procesar

In [9]:
model_0 = Sequential()
model_0.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_0.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [10]:
model_0_checkpoint_callback = ModelCheckpoint(
    filepath=join(checkpoints_path, 'basic'),
    save_weights_only=True,
    monitor='val_auc',
    mode='max',
    save_best_only=True)

In [11]:
%%time
history = model_0.fit(x_train, y_train, epochs=50, 
            validation_data=(x_valid, y_valid),
            callbacks=[model_0_checkpoint_callback],
            verbose=0)
# Cargo el mejor modelo entrenado
model_0.load_weights(join(checkpoints_path, 'basic'))
verify_model(model_0, x_train, y_train, x_valid, y_valid)

Wall time: 3.33 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.697809,0.760684,0.546798,0.569231,0.743733
1,Validacion,0.673187,0.647887,0.703704,0.431818,0.851852


### Reemplazando datos invalidos

In [12]:
x_train_nonzero, _data = replace_outliers_zeros(x_train, {}, zeros, mean_median=True)
x_test_nonzero, _data = replace_outliers_zeros(x_test, {}, zeros, mean_median=True, data_to_replace=_data)
x_valid_nonzero, _data = replace_outliers_zeros(x_valid, {}, zeros, mean_median=True, data_to_replace=_data)

In [13]:
model_1 = Sequential()
model_1.add(Dense(units=1, activation='sigmoid', input_shape=(x_train.shape[1],)))
model_1.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_1_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'non_zero'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [14]:
%%time 
history = model_1.fit(x_train_nonzero, y_train, epochs=50, validation_data=(x_valid_nonzero, y_valid), callbacks=[model_1_checkpoint_callback], verbose=0)
# Cargo el mejor modelo entrenado
model_1.load_weights(join(checkpoints_path, 'non_zero'))
verify_model(model_1, x_train_nonzero, y_train, x_valid_nonzero, y_valid)

KeyboardInterrupt: 

Se puede observar como en ambos casos se obtuvo un mejor resultado en validacion que en train, lo cual es un comportamiento que se puede considerar poco esperado

### Reemplazando outlayers

In [15]:
x_train_clean, _data = replace_outliers_zeros(x_train, outlayers, zeros, mean_median=True)
x_test_clean, _data = replace_outliers_zeros(x_test, outlayers, zeros, mean_median=True, data_to_replace=_data)
x_valid_clean, _data = replace_outliers_zeros(x_valid, outlayers, zeros, mean_median=True, data_to_replace=_data)

In [16]:
model_2 = Sequential()
model_2.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_clean.shape[1],)))
model_2.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_2_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'clean'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [17]:
%%time 
history = model_2.fit(x_train_clean, y_train, epochs=50, validation_data=(x_valid_clean, y_valid), callbacks=[model_2_checkpoint_callback], verbose=0)
# Cargo el mejor modelo entrenado
model_2.load_weights(join(checkpoints_path, 'clean'))
verify_model(model_2, x_train_clean, y_train, x_valid_clean, y_valid)

Wall time: 3.41 s


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.665411,0.487179,0.753695,0.459459,0.773756
1,Validacion,0.686228,0.436620,0.925926,0.384615,0.939394


### Normalizando los datos de entrada

In [18]:
x_train_norm, _norm_dict = normalize(x_train_clean, None)
x_valid_norm, _norm_dict = normalize(x_valid_clean, _norm_dict)
x_test_norm, _norm_dict = normalize(x_test_clean, _norm_dict)

In [19]:
model_3 = Sequential()
model_3.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
model_3.compile(optimizer=SGD(), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

model_3_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'norm'),
                                              save_weights_only=True,
                                              monitor='val_auc',
                                              mode='max',
                                              save_best_only=True)

In [20]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(join(checkpoints_path, 'norm'))
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 1s 29ms/step - loss: 0.7517 - auc: 0.5624 - val_loss: 0.7721 - val_auc: 0.5102
Epoch 2/100
18/18 [==============================] - 0s 13ms/step - loss: 0.7294 - auc: 0.5857 - val_loss: 0.7545 - val_auc: 0.5326
Epoch 3/100
18/18 [==============================] - 0s 15ms/step - loss: 0.7096 - auc: 0.6079 - val_loss: 0.7383 - val_auc: 0.5540
Epoch 4/100
18/18 [==============================] - 0s 13ms/step - loss: 0.6913 - auc: 0.6295 - val_loss: 0.7235 - val_auc: 0.5699
Epoch 5/100
18/18 [==============================] - 0s 13ms/step - loss: 0.6746 - auc: 0.6496 - val_loss: 0.7104 - val_auc: 0.5811
Epoch 6/100
18/18 [==============================] - 0s 13ms/step - loss: 0.6597 - auc: 0.6673 - val_loss: 0.6984 - val_auc: 0.5931
Epoch 7/100
18/18 [==============================] - 0s 19ms/step - loss: 0.6465 - auc: 0.6828 - val_loss: 0.6877 - val_auc: 0.5991
Epoch 8/100
18/18 [==============================] - 0s 12ms/step - loss: 0.

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.842757,0.857550,0.610837,0.712644,0.792105
1,Validacion,0.755347,0.830986,0.444444,0.500000,0.797297


### Early Stopping
Los hyperparámetrso principales de este callback son:
- monitor: indica la variable a monitorear
- min_delta: la minima diferencia que se considera como mejora
- patience: cantidad de epochs sin mejoras antes de parar

In [21]:
stop_early = EarlyStopping(monitor='val_auc', patience=20)

In [22]:
%%time 
history = model_3.fit(x_train_norm, y_train, epochs=50, validation_data=(x_valid_norm, y_valid), callbacks=[model_3_checkpoint_callback, stop_early], verbose=1)
# Cargo el mejor modelo entrenado
model_3.load_weights(join(checkpoints_path, 'norm'))
verify_model(model_3, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/50
18/18 [==============================] - 0s 19ms/step - loss: 0.4723 - auc: 0.8425 - val_loss: 0.5237 - val_auc: 0.7559
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.4722 - auc: 0.8427 - val_loss: 0.5235 - val_auc: 0.7553
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 0.4720 - auc: 0.8432 - val_loss: 0.5235 - val_auc: 0.7553
Epoch 4/50
18/18 [==============================] - 0s 13ms/step - loss: 0.4720 - auc: 0.8430 - val_loss: 0.5235 - val_auc: 0.7561
Epoch 5/50
18/18 [==============================] - 0s 14ms/step - loss: 0.4719 - auc: 0.8431 - val_loss: 0.5231 - val_auc: 0.7564
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 0.4717 - auc: 0.8428 - val_loss: 0.5230 - val_auc: 0.7561
Epoch 7/50
18/18 [==============================] - 0s 12ms/step - loss: 0.4717 - auc: 0.8430 - val_loss: 0.5229 - val_auc: 0.7580
Epoch 8/50
18/18 [==============================] - 0s 14ms/step - loss: 0.4715 - auc:

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.843501,0.866097,0.605911,0.723529,0.791667
1,Validacion,0.763172,0.845070,0.444444,0.521739,0.800000


Se puede evidenciar como al utilizar early stopping el entrenamiento finaliza antes debido a que las mejoras son reducidas

## Ajuste de Hyperparámetros

### Learning rate
Por defecto en SGD el learning rate que utiliza keras es 0.01

In [23]:
learning_rates = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]
learning_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'learning'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in learning_rates:
    learning = Sequential()
    learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    learning.compile(optimizer=SGD(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[learning_checkpoint_callback, stop_early], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    learning.load_weights(join(checkpoints_path, 'learning'))
    metrics = verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.535211,0.535211,0.740741,0.740741,0.75952,0.75952,0.763172,0.765258,0.768388,0.768388,0.779864,0.779864,0.779864,0.779864,0.786124,0.786124,0.786124
1,Epochs,20.000000,20.000000,21.000000,20.000000,20.00000,20.00000,20.000000,20.000000,26.000000,20.000000,41.000000,36.000000,26.000000,33.000000,40.000000,30.000000,40.000000


Es interesante observar como para los learning rate mas chicos, el entrenamiento se corta debido al early stoping en los primeros 15 epochs, ya que dentro de esos primeros 15 las mejoras son pocas debido a su avance "suave", en comparación con los learning rates mas altos que obtienen una mejor optimización en mayor cantidad de epochs

In [24]:
# Cargo el mejor modelo entrenado
learning.load_weights(join(checkpoints_path, 'learning'))
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.798366,0.834758,0.561576,0.662791,0.767016
1,Validacion,0.786124,0.788732,0.555556,0.500000,0.823529


#### Scheduling learning rate

In [25]:
learning_schedule = ExponentialDecay(initial_learning_rate=2.7, decay_steps=100, decay_rate=0.96)
exponential_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'exponential'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)


In [26]:
learning = Sequential()
learning.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
learning.compile(optimizer=SGD(learning_rate=learning_schedule), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

In [27]:
history = learning.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[exponential_checkpoint_callback, stop_early], 
                      verbose=1)
# Cargo el mejor modelo entrenado
learning.load_weights(join(checkpoints_path, 'exponential'))
verify_model(learning, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 1s 29ms/step - loss: 0.5443 - auc: 0.7949 - val_loss: 0.7662 - val_auc: 0.7527
Epoch 2/100
18/18 [==============================] - 0s 14ms/step - loss: 0.5255 - auc: 0.8146 - val_loss: 0.7111 - val_auc: 0.7079
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5087 - auc: 0.8205 - val_loss: 0.6741 - val_auc: 0.6964
Epoch 4/100
18/18 [==============================] - 0s 19ms/step - loss: 0.5069 - auc: 0.8274 - val_loss: 0.5795 - val_auc: 0.7796
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 0.5188 - auc: 0.8182 - val_loss: 0.5710 - val_auc: 0.7092
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5211 - auc: 0.8103 - val_loss: 0.6702 - val_auc: 0.6753
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5497 - auc: 0.7982 - val_loss: 0.6023 - val_auc: 0.7415
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5118 

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.805117,0.752137,0.660099,0.606335,0.792793
1,Validacion,0.779864,0.802817,0.629630,0.548387,0.850746


### Momentum

In [28]:
momentum_arr = [0.5, 0.6, 0.85, 0.9, 0.95, 0.99, 0.999]

arr_of_metrics = []
auc_res = {'momentum':['AUC', 'Epochs']}
for i in momentum_arr:
    momentum_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'momentum_{}'.format(i)),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    
    mom_model = Sequential()
    mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback], 
                          verbose=1)
    # Cargo el mejor modelo entrenado
    mom_model.load_weights(join(checkpoints_path, 'momentum_{}'.format(i)))
    metrics = verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

,momentum,0.5,0.6,0.85,0.9,0.95,0.99,0.999
0,AUC,0.779343,0.790297,0.787167,0.788211,0.776995,0.76578,0.74361
1,Epochs,99.000000,99.000000,99.000000,99.000000,99.000000,99.00000,99.00000


In [29]:
momentum_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'momentum'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

mom_model = Sequential()
mom_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
mom_model.compile(optimizer=SGD(learning_rate=learning_schedule, momentum=0.6), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = mom_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[momentum_checkpoint_callback, stop_early], 
                          verbose=1)
# Cargo el mejor modelo entrenado
mom_model.load_weights(join(checkpoints_path, 'momentum'))
verify_model(mom_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 0s 27ms/step - loss: 0.6346 - auc: 0.7885 - val_loss: 0.6822 - val_auc: 0.7243
Epoch 2/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5538 - auc: 0.8041 - val_loss: 0.6052 - val_auc: 0.7493
Epoch 3/100
18/18 [==============================] - 0s 13ms/step - loss: 0.5783 - auc: 0.7975 - val_loss: 0.8669 - val_auc: 0.7551
Epoch 4/100
18/18 [==============================] - 0s 13ms/step - loss: 0.6356 - auc: 0.7840 - val_loss: 0.6859 - val_auc: 0.7598
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 0.6056 - auc: 0.7885 - val_loss: 0.6429 - val_auc: 0.6774
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5885 - auc: 0.7936 - val_loss: 0.5778 - val_auc: 0.7449
Epoch 7/100
18/18 [==============================] - 0s 2ms/step - loss: 0.5709 - auc: 0.7961 - val_loss: 0.6204 - val_auc: 0.7136
Epoch 8/100
18/18 [==============================] - 0s 12ms/step - loss: 0.550

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.836863,0.809117,0.694581,0.677885,0.820809
1,Validacion,0.767866,0.774648,0.555556,0.483871,0.820896


### Optimizador Adam

In [30]:
%%time
adam_lr = [1e-5, 1e-4, 1e-3, 0.01, 0.05, 0.1, 0.22, 0.33, 0.47, 0.56, 0.81, 1.2, 1.8, 2.2, 2.7, 3.3, 4.7]

arr_of_metrics = []
auc_res = {'L_r':['AUC', 'Epochs']}
for i in adam_lr:
    adam_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'adam_{}'.format(i)),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)
    adam_mod = Sequential()
    adam_mod.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
    adam_mod.compile(optimizer=Adam(learning_rate=i), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])

    history = adam_mod.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback], 
                          verbose=1, use_multiprocessing=True)
    # Cargo el mejor modelo entrenado
    adam_mod.load_weights(join(checkpoints_path, 'adam_{}'.format(i)))
    metrics = verify_model(adam_mod, x_train_norm, y_train, x_valid_norm, y_valid)
    arr_of_metrics.append(metrics)
    auc_res[i] = [metrics['AUC ROC'][1], history.epoch[-1]]
    clear_output(wait=True)
pd.DataFrame(data=auc_res)

Wall time: 2min 1s


,L_r,1e-05,0.0001,0.001,0.01,0.05,0.1,0.22,0.33,0.47,0.56,0.81,1.2,1.8,2.2,2.7,3.3,4.7
0,AUC,0.707877,0.53469,0.726135,0.764737,0.769431,0.770996,0.771518,0.783516,0.778299,0.780908,0.798122,0.786124,0.781429,0.788732,0.784559,0.791862,0.768388
1,Epochs,99.000000,99.00000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000


In [31]:
%%time
adam_checkpoint_callback = ModelCheckpoint(filepath=join(checkpoints_path, 'adam'),
                                                  save_weights_only=True,
                                                  monitor='val_auc',
                                                  mode='max',
                                                  save_best_only=True)

adam_model = Sequential()
adam_model.add(Dense(units=1, activation='sigmoid', input_shape=(x_train_norm.shape[1],)))
adam_model.compile(optimizer=Adam(learning_rate=4.7), loss=BinaryCrossentropy(), metrics=[AUC(name='auc')])
history = adam_model.fit(x_train_norm, y_train, epochs=100, validation_data=(x_valid_norm, y_valid), callbacks=[adam_checkpoint_callback], 
                          verbose=1)
# Cargo el mejor modelo entrenado
adam_model.load_weights(join(checkpoints_path, 'adam'))
verify_model(adam_model, x_train_norm, y_train, x_valid_norm, y_valid)

Epoch 1/100
18/18 [==============================] - 0s 23ms/step - loss: 2.2540 - auc: 0.7467 - val_loss: 2.6254 - val_auc: 0.3625
Epoch 2/100
18/18 [==============================] - 0s 10ms/step - loss: 1.9885 - auc: 0.7084 - val_loss: 2.1503 - val_auc: 0.5540
Epoch 3/100
18/18 [==============================] - 0s 11ms/step - loss: 1.9105 - auc: 0.7326 - val_loss: 1.2407 - val_auc: 0.6635
Epoch 4/100
18/18 [==============================] - 0s 16ms/step - loss: 1.1210 - auc: 0.7123 - val_loss: 2.1823 - val_auc: 0.7167
Epoch 5/100
18/18 [==============================] - 0s 2ms/step - loss: 1.5035 - auc: 0.7256 - val_loss: 0.8424 - val_auc: 0.6881
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 1.0909 - auc: 0.7514 - val_loss: 1.8338 - val_auc: 0.5545
Epoch 7/100
18/18 [==============================] - 0s 14ms/step - loss: 1.8002 - auc: 0.7324 - val_loss: 1.7305 - val_auc: 0.7619
Epoch 8/100
18/18 [==============================] - 0s 2ms/step - loss: 1.551

,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.782578,0.772080,0.640394,0.619048,0.787791
1,Validacion,0.773083,0.704225,0.629630,0.447368,0.833333


Evaluación contra test

In [32]:
print('Utilizando SGD')
verify_model(mom_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando SGD


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.836863,0.809117,0.694581,0.677885,0.820809
1,Test,0.861336,0.794872,0.657895,0.609756,0.826667


In [33]:
print('Utilizando Adam')
verify_model(adam_model, x_train_norm, y_train, x_test_norm, y_test, valid_label='Test')

Utilizando Adam


,Set,AUC ROC,Especificidad,Sensibilidad,Valor Predictivo Positivo,Valor Predictivo Negativo
0,Train,0.782578,0.772080,0.640394,0.619048,0.787791
1,Test,0.859987,0.820513,0.684211,0.650000,0.842105


Finalmente se obtiene que el modelo que utiliza como optimizador SGD y los hyperparámetros optimizados fue el mejor de los entrenados hasta ahora

Cabe destacar una particularidad, que los resustados en Test son mejores que en el set de validacion, e incluso mejores que el set de train, con lo cual concluir que la metrica de nuestro modelo es AUC=0.86 no sería correcto, ya que es el caso en el que tenemos un estimador con alta varianza, por lo tanto una estimación utilizando k-folding sería mas adecuada (si hay tiempo la hago...)